In [64]:
from testing_criterions import CPTDigraphManager
from diblob import DigraphManager


def minimal_test_cases_without_cost(digraph_manager, source='s', sink='t'):

    if source in digraph_manager or sink in digraph_manager:
        raise Exception('sink/source should be unique over digraph namespace!')
    
    dict_json_representation = dict(digraph_manager(digraph_manager.root_diblob_id))
    cpt = CPTDigraphManager(dict_json_representation, cost_function={})


    minimal_elements, maximal_elements = cpt.get_edge_elements()
    
    cpt.add_nodes(source, sink)

    minimal_edges = [(source, node_id) for node_id in minimal_elements]
    maximal_edges = [(node_id, sink) for node_id in maximal_elements]
    
    if not minimal_edges or not maximal_edges:
        raise Exception('Should be at leas minimal/maximal element in the diblob!')
    
    new_edges = minimal_edges + maximal_edges + [(sink, source)]

    cpt.connect_nodes(*new_edges)

    weight = len(cpt.nodes) * (len(cpt.nodes) + 1) / 2
    cost_function_update_dict = {edge_id: weight for edge_id in cpt.edges}
    cost_function_update_dict[(sink, source)] = 1

    cpt.update_cost_function(cost_function_update_dict)
    return cpt.compute_cpt(start_node=source)


def extract_test_cases(test_cases, delimiter = ',', source='s', sink='t'):
    test_cases_string = ','.join([elem[0] for elem in test_cases])
    test_cases = test_cases_string.split(f'{sink},{source}')
    test_cases = [f"{source},{test_case.replace(source, '').replace(sink, '')[1:-1]},{sink}".replace(',', delimiter) 
                  for test_case in test_cases]
    return test_cases
    

In [65]:
cpt_digraph_manager = DigraphManager({"B0": {}})
cpt_digraph_manager.add_nodes('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I')
cpt_digraph_manager.connect_nodes(('A', 'C'), ('B', 'C'), ('C', 'D'), ('C', 'E'), ('C', 'F'),
                       ('D', 'I'), ('D', 'G'), ('E', 'G'), ('E', 'H'), ('F', 'H'))

test_cases, cost = minimal_test_cases_without_cost(cpt_digraph_manager, source='S', sink='T')

In [66]:
print(extract_test_cases(test_cases, delimiter='|', source='S', sink='T'))

['S|B|C|F|H|T', 'S|A|C|D|G|T', 'S|A|C|E|H|T', 'S|A|C|E|G|T', 'S|A|C|D|I|T']
